In [ ]:
import os
import time

import requests
import pandas as pd
import numpy as np
from tqdm import tqdm


access_token = os.get["vk_access_token"]

In [6]:
PUBLIC_NAME = "vso_1"  # "hbtvv" # "2oprosa" # "public180811191" # "vso_1"
OUTPUT_FILE = "important_polls1.csv"

In [3]:
response = requests.post(
    "https://api.vk.com/method/wall.get",
    params={
        "domain": PUBLIC_NAME,
        "v": "5.199",
        "access_token": access_token,
        "count": 1,
        "offset": 0,
    },
)

In [4]:
num_posts = response.json()["response"]["count"]
num_posts

23554

In [5]:
data = {"id": [], "text": [], "question": [], "answer1": [], "answer2": [], "rate1": [], "rate2": [], "votes1" : [], "votes2" : []}
step = 100
for offset in tqdm(range(0, num_posts, step)):
    response = requests.post(
        "https://api.vk.com/method/wall.get",
        params={
            "domain": PUBLIC_NAME,
            "v": "5.199",
            "access_token": access_token,
            "count": step,
            "offset": offset,
        },
    )
    if response.status_code != 200:
        print(f"on {offset=} error occured")
        break
    posts = response.json()["response"]["items"]

    for post in posts:
        for attachment in post["attachments"]:
            if "poll" in attachment:
                poll = attachment["poll"]
                answers = poll["answers"]
                if len(answers) < 2:
                    continue
                if len(answers) > 2:
                    answers.sort(key=lambda x: x["rate"], reverse=True)
                    answers = answers[:2]
                data["id"].append(poll["id"])
                data["text"].append(post["text"])
                data["question"].append(poll["question"])
                for i, answer in enumerate(answers):
                    data[f"answer{i+1}"].append(answer["text"])
                    data[f"votes{i+1}"].append(answer["votes"])
                    data[f"rate{i+1}"].append(answer["rate"])
    time.sleep(np.random.random() * 0.8)

100%|██████████| 236/236 [08:19<00:00,  2.12s/it]


In [7]:
df = pd.DataFrame(data)
df.to_csv(OUTPUT_FILE, index=False)

In [8]:
df = pd.read_csv(OUTPUT_FILE)
df

,id,text,question,answer1,answer2,rate1,rate2,votes1,votes2
0,885598345,Хотите ли вы детей?,Что бы вы выбрали?,Да,Нет,51.74,48.26,6926,6461
1,966697749,Вы коллекционер монет. Однажды на барахолке по...,Что бы вы выбрали?,Продать монету. Друг будет всю жизнь считать м...,"Признаться, что монета поддельная",38.45,61.55,809,1295
2,967001095,Вы изобрели революционное противозачаточное. В...,Что бы Вы выбрали?,"Согласиться. Дама внушает доверие, но неизвест...",Отказаться и продолжить искать подопытных само...,86.80,13.20,848,129
3,966790489,"Представим, что вас крупно подставили таким об...",Что бы вы выбрали?,Отдать мошеннику миллион рублей.,"Не давать мошеннику денег, но потратить на суд...",28.62,71.38,417,1040
4,965896595,Вам заплатят 1 млн$ если вы переночуете в 100 ...,Что бы вы выбрали?,Согласиться,Отказаться,88.96,11.04,2344,291
...,...,...,...,...,...,...,...,...,...
22770,165384516,Что бы вы выбрали?,Что бы вы выбрали?,всякий раз получаться невидимым на всех фотогр...,всегда пахнуть куриной котлетой,71.25,28.75,4205,1697
22771,165383638,Что бы вы выбрали?,Что бы вы выбрали?,кисть правой руки в виде одного большого пальца,одну густую бровь в центре лба,16.45,83.55,598,3037
22772,165382056,Что бы вы выбрали?,Что бы вы выбрали?,"психологическое расстройство, ввиду которого в...",полное отсутствие волос на голове,45.57,54.43,1966,2348
22773,165381714,Что бы вы выбрали?,Что бы вы выбрали?,"наличие правой руки, длиною до колена","очень маленький левый глаз, размером в однокоп...",66.96,33.04,2809,1386
